## Run this first if you want NDVI/NDWI basemaps

In [2]:
import ee
ee.Authenticate()


True

## Vibes

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(filename='geovibes.log', level=logging.ERROR)

try:
    from src.ui import GeoVibes
    vibes = GeoVibes.from_config('config/resnet_config.json', verbose=True)
except Exception as e:
    logging.error(f"Jupyter kernel crashed: {str(e)}", exc_info=True)
    raise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Earth Engine initialized with user credentials


ValueError: No database available - either duckdb_path or duckdb_directory must be provided